In [ ]:
# Install required packages
!pip install diffusers transformers xformers git+https://github.com/huggingface/accelerate.git
!pip install opencv-contrib-python controlnet_aux google-generativeai gradio

In [ ]:
# Import required libraries
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import OpenposeDetector
import torch
from PIL import Image
import numpy as np
import cv2
from google.colab import files
from io import BytesIO
import google.generativeai as genai
import gradio as gr

/usr/local/lib/python3.11/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [ ]:
key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=key)

In [1]:
test_prompt = "best tupac one liner."
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content([test_prompt])
print("Gemini response:", response.text.strip())

NameError: name 'genai' is not defined

In [ ]:
# Enable GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize ControlNet model with optimizations
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16
)

# Initialize pipeline with optimizations
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
)

# Configure pipeline for optimal performance
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()

Using device: cuda


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
import tempfile
import shutil

def save_uploaded_files(file_objs):
    image_paths = []

    try:
        for file_obj in file_objs:
            temp_path = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False).name
            with open(file_obj.name, "rb") as src:
                with open(temp_path, "wb") as dst:
                    shutil.copyfileobj(src, dst)
            image_paths.append(temp_path)

        print("Saved all files:", image_paths)
        return image_paths

    except Exception as e:
        print("Error saving files:", str(e))
        return []

def analyze_from_uploaded_files(file_objs):
    image_paths = []

    try:
        for file_obj in file_objs:
            temp_path = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False).name
            with open(file_obj.name, "rb") as src:
                with open(temp_path, "wb") as dst:
                    shutil.copyfileobj(src, dst)
            image_paths.append(temp_path)

        print("Saved all files:", image_paths)

        # Analyze images via Gemini
        analysis = analyze_images(image_paths)
        print("Got analysis:", analysis)

        return json.dumps(analysis, indent=2), analysis

    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"❌ Error: {str(e)}", {}

    finally:
        for path in image_paths:
            if os.path.exists(path):
                os.remove(path)



# 🧠 Analyze images using Gemini Vision
import os
import uuid
from typing import Dict, List, Union
import json

def analyze_images(image_paths: List[str]) -> Dict[str, Union[Dict, str]]:
    """Generate detailed tags for the uploaded images using Gemini Vision"""
    model = genai.GenerativeModel('gemini-2.0-flash')
    image_tags = {}
    print('hey there from analyze_images')
    for path in image_paths:
        print('path' + path)
        try:
            with open(path, "rb") as img_file:
                image_data = img_file.read()

            prompt = [
                "Analyze this fabric image and provide detailed information about: "
                "material composition, color palette (including secondary colors), "
                "pattern characteristics, texture properties, and structural elements. "
                "Format as JSON with separate sections for each characteristic.",
                {"mime_type": "image/jpeg", "data": image_data}
            ]

            response = model.generate_content(prompt)
            response_text = response.text.strip()

            # Clean the response text by removing extra formatting
            response_text = response_text.replace('```json\n', '').rstrip('```')

            # Validate response
            if not response_text:
                print(f"Warning: Empty response from Gemini for {path}")
                image_tags[os.path.basename(path)] = {"error": "Empty response from API"}
                continue

            try:
                # Parse the response text into a proper JSON structure
                parsed_response = json.loads(response_text)

                result = {
                    'color_palette': parsed_response.get('color_palette', {}).get('primary_color', 'unknown'),
                    'material_type': parsed_response.get('material_composition', {}).get('likely_material', 'unknown'),
                    'pattern': parsed_response.get('pattern_characteristics', {}).get('pattern_type', 'unknown'),
                    'texture': parsed_response.get('texture_properties', {}).get('visual_texture', 'unknown'),
                    'structural_elements': parsed_response.get('structural_elements', {}).get('details', [])
                }

                filename = os.path.basename(path)
                image_tags[filename] = result

            except json.JSONDecodeError as e:
                print(f"Error parsing JSON for {path}: {str(e)}")
                print(f"Raw response: {response_text}")
                image_tags[os.path.basename(path)] = {"error": f"Invalid JSON response: {str(e)}"}

            except Exception as e:
                print(f"Error processing {path}: {str(e)}")
                image_tags[os.path.basename(path)] = {"error": f"Processing error: {str(e)}"}

        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
            image_tags[os.path.basename(path)] = {"error": str(e)}

    return image_tags


# 🧵 Generate design prompt from fabric analysis
def create_design_prompt(image_tags):
    """Generate a creative prompt for fusing fabric elements into an upcycled design"""
    model = genai.GenerativeModel('gemini-2.0-flash')

    fabric_descriptions = []
    for filename, tag_data in image_tags.items():
        try:
            if isinstance(tag_data, dict) and 'error' not in tag_data:
                # Use color_palette instead of colors
                colors = tag_data.get('color_palette', 'unknown')
                if isinstance(colors, list):
                    color_str = ', '.join(colors)
                else:
                    color_str = colors

                desc = f"{filename}: {color_str} {tag_data['material_type']} "
                desc += f"featuring {tag_data['pattern']} patterns and {tag_data['texture']} texture"
                fabric_descriptions.append(desc)

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            fabric_descriptions.append(f"{filename}: (unreadable tags)")

        prompt = (
                "Design a practical, wearable garment that combines these materials in a simple, realstic and functional way. "
                "Focus on everyday wearability and comfort.DOES not need to be a artistic piece and DO NOT write a full exploration of themes or create an outfit"
                "Think about how these fabrics can work together in a straightforward, practical design please keep the concept realistic and not more than 300 words.\n\n"

                "Materials:\n" +
                "\n".join(fabric_descriptions) + "\n\n"

                "Design a single simple, practical garment NOT an outfit that combines these elements in a functional way"
                "MAX 150 TOKEN"
            )
    print("Getting creative concept from Gemini...")
        # Use lower temperature for more focused responses
    generation_config = {
        'temperature': 0.2,  # Lower temperature for more focused responses
        #  'max_tokens': 150  # need to understand this
    }

    response = model.generate_content([prompt], generation_config=generation_config)
    creative_concept = response.text.strip()
    return creative_concept

# 🎨 Generate final image
def final_design_wrapper(prompt, files):
    """
    Wraps your generate_final_design to handle Gradio File objects.
    """
    image_paths = save_uploaded_files(files)  # same method as before

      # 2. Blend them into a control image
    control_image = process_and_blend_images(image_paths)


    return generate_design(prompt, [control_image])[0]


  # Helper function for image processing
def process_and_blend_images(image_paths, size=(512, 512)):
    """
    Process multiple images and blend their Canny edge maps into one control image.
    """
    edge_accumulator = None
    for path in image_paths:
        image = Image.open(path).resize(size).convert("RGB")
        np_image = np.array(image)
        edges = cv2.Canny(np_image, 100, 200)
        edges_rgb = np.stack([edges]*3, axis=-1)  # Convert to RGB-like array

        if edge_accumulator is None:
            edge_accumulator = edges_rgb.astype(np.float32)
        else:
            edge_accumulator += edges_rgb.astype(np.float32)

    # Average edges
    averaged_edges = (edge_accumulator / len(image_paths)).astype(np.uint8)
    control_image = Image.fromarray(averaged_edges).convert("RGB")
    return control_image

def generate_design(prompt, control_images, num_steps=30, guidance_scale=7.5): # num steps reduced from 30 --> 5
    """
    Generate designs using the ControlNet pipeline for multiple control images.

    Args:
        prompt (str): The text prompt to guide generation.
        control_images (List[PIL.Image]): List of processed control images.
        num_steps (int): Number of inference steps.
        guidance_scale (float): Classifier-free guidance scale.

    Returns:
        List[PIL.Image]: Generated images for each control image.
    """
    results = []
    for i, control_image in enumerate(control_images):
        print(f"Generating image {i+1}/{len(control_images)}...")
        result = pipe(
            prompt=prompt,
            image=control_image,
            num_inference_steps=num_steps,
            guidance_scale=guidance_scale
        ).images[0]
        results.append(result)
    return results

In [ ]:

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 🧥 Fabric Fusion Designer")

    with gr.Row():
        image_input = gr.File(
            file_types=["image"],
            label="Upload Fabric Images",
            file_count="multiple"
        )
        analyze_button = gr.Button("Analyze Fabrics 🧠")

    analysis_output = gr.Textbox(label="📝 Fabric Analysis JSON", lines=15)
    prompt_output = gr.Textbox(label="🧵 Generated Design Prompt", lines=4)
    prompt_state = gr.State()
    analysis_state = gr.State()

    def load_images_from_files(files):
        return [Image.open(file) for file in files]

    analyze_button.click(
        fn=analyze_from_uploaded_files,
        inputs=image_input,
        outputs=[analysis_output, analysis_state]
    )

    generate_prompt_button = gr.Button("Generate Prompt from Analysis 🧶")
    generate_prompt_button.click(
        create_design_prompt,
        inputs=analysis_state,
        outputs=[prompt_output]
    ).then(fn=lambda x: x, inputs=prompt_output, outputs=prompt_state)

    design_button = gr.Button("Generate Final Garment Design 🎨")
    design_output = gr.Image(label="Final Design", type="pil")
    design_button.click(
        fn=final_design_wrapper,
        inputs=[prompt_state, image_input],
        outputs=design_output
    )

demo.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e0fc3fcc04668950ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Saved all files: ['/tmp/tmpuc38bewe.jpg', '/tmp/tmphs86pgnt.jpg', '/tmp/tmpmn2un74k.jpg']
hey there from analyze_images
path/tmp/tmpuc38bewe.jpg
path/tmp/tmphs86pgnt.jpg
path/tmp/tmpmn2un74k.jpg
Got analysis: {'tmpuc38bewe.jpg': {'color_palette': 'Dark Navy Blue', 'material_type': 'Based on the appearance and the use case (athletic pants), the material is likely a blend of polyester and cotton or a synthetic fabric like nylon or a polyester blend. The presence of an elastic waistband further suggests a blend that allows for stretch and recovery. Without microscopic analysis, it is difficult to say the exact blend.', 'pattern': 'Solid color with accent stripe and logo', 'texture': 'unknown', 'structural_elements': []}, 'tmphs86pgnt.jpg': {'color_palette': 'unknown', 'material_type': 'unknown', 'pattern': 'Color Blocking, Chevron', 'texture': 'unknown', 'structural_elements': []}, 'tmpmn2un74k.jpg': {'color_palette': 'Navy Blue', 'material_type': 'unknown', 'pattern': 'Solid with Accent 

Token indices sequence length is longer than the specified maximum sequence length for this model (149 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['mn 2 un 7 4 k . jpg ) trims the cuffs and hem , creating a cohesive design element . the jacket features a zip front closure , simple welt pockets , and a relaxed fit for comfort and ease of movement . the polyester / cotton blend of the main body ensures durability and breathability , while the other fabrics add style without sacrificing practicality .']


Generating image 1/1...


  0%|          | 0/30 [00:00<?, ?it/s]